In [13]:
import numpy as np

n = 12 #Sequence length
m = 4 #Chunk length
k = 2 #Amount of neighbours
r = 5 #Retrieval length
d = 2 #Embedding size
l = n // m #Number of chunks

# Parameters
Q = np.random.rand(d, d)
K = np.random.rand(d, d)
V = np.random.rand(d, d)

def cross_attention(chunk, neighbour):
    #print(hash(str(chunk)), chunk.shape, hash(str(neighbour)), neighbour.shape)
    #print('--')
    m, d = chunk.shape
    r, d = neighbour.shape
    queries = chunk @ Q
    keys = neighbour @ K
    logits = queries @ keys.T
    values = neighbour @ V
    return logits, values

def multi_neighbour_cross_attention(chunk, neighbours):
    m, d = chunk.shape
    k, r, d = neighbours.shape
    #print(chunk.shape, neighbours.shape)
    #print('***')
    logits, values = np.vectorize(cross_attention, signature='(m,d),(r,d)->(m,r),(r,d)')(chunk, neighbours)
    print(logits)
    #print(logits.shape, values.shape)
    assert logits.shape == (k, m, r)
    assert values.shape == (k, r, d)
    #print(logits.shape, values.shape)
    #logits += relative_positional_encodings(m, r)[None, :, :]
    logits = np.moveaxis(logits, 0, -1).reshape((m, r * k))
    values = np.moveaxis(values, 0, 1).reshape((r * k, d))
    #print(logits.shape)
    #print(values.shape)
    print('_____________')
    return logits @ values

def multi_chunk_cross_attention(observation, neighbours):
    attending_chunks = np.pad(observation[m-1:],((0, m - 1), (0, 0)),mode='constant').reshape(l, m, d)
    #print(attending_chunks.shape, neighbours.shape)
    #print('.....')
    chunked_output = np.vectorize(multi_neighbour_cross_attention,signature='(m,d),(k,r,d)->(m,d)')(attending_chunks, neighbours)
    #print(chunked_output.shape)
    #print('&&&&&')
    assert chunked_output.shape == (l, m, d)
    output = np.pad(chunked_output.reshape(n, d),((m - 1, 0), (0, 0)), mode='constant')[:n]
    return output

observation = np.random.rand(n, d) # Input
print(hash(str(observation)))

neighbours = np.random.rand(l, k, r, d)
print(hash(str(neighbours)))

print('++++++++++++++')

h = multi_chunk_cross_attention(observation, neighbours)

assert h.shape == (n, d) # Output

#print(h)

-4237097945216324929
839605620643339064
++++++++++++++
[[[0.36438333 0.36111088 0.18909095 0.33910186 0.39113616]
  [0.34504458 0.34396399 0.1752655  0.32365606 0.37735245]
  [0.3100022  0.307685   0.159994   0.28908395 0.3343759 ]
  [0.43178853 0.42726507 0.22528229 0.4010142  0.4612588 ]]

 [[0.42720214 0.4788569  0.44210532 0.36674087 0.53541341]
  [0.41372638 0.44055016 0.41639913 0.33506078 0.50248024]
  [0.36557337 0.40440911 0.37560608 0.30918192 0.45446247]
  [0.50328553 0.57156254 0.52460531 0.43849043 0.63590171]]]
_____________
[[[0.53777186 0.47054536 0.20969069 0.60461306 0.49281796]
  [0.5439917  0.47553652 0.20984378 0.61171768 0.49580425]
  [0.55745884 0.4863003  0.20995849 0.6271112  0.50201127]
  [0.41213934 0.35915855 0.15335252 0.46372668 0.36890839]]

 [[0.78388561 0.56371234 0.49704147 0.68017828 0.59470315]
  [0.79053495 0.57014531 0.49885353 0.68552355 0.60052465]
  [0.8047015  0.58406564 0.50240144 0.69685655 0.6130285 ]
  [0.59293727 0.43173857 0.36818833 0.51

In [18]:
import numpy as np
from einops import rearrange

n = 12 #Sequence length
m = 4 #Chunk length
k = 2 #Amount of neighbours
r = 5 #Retrieval length
d = 2 #Embedding size
l = n // m #Number of chunks

#observation = np.random.rand(n, d)
print(hash(str(observation)))

#Get k-nearest neighbours
chunks = rearrange(observation, '(l m) d -> l m d', l=l)
#print(chunks.shape)

n_chunk = chunks[:-1]
#print(n_chunk.shape)

#neighbours = np.random.rand(l, k, r, d)
print(hash(str(neighbours)))

#Chunk cross attention
attending_chunks = np.pad(observation[m-1:], ((0, m - 1), (0, 0)), mode='constant').reshape(l, m, d)
#print(attending_chunks.shape)

print('--------')
for u in range(l):
    chunk = chunks[u]
    neighbour = neighbours[u]
    logits = []
    values = []
    for n in neighbour:
        logit, value = cross_attention(chunk, n)
        logits.append(logit)
        values.append(value)
    logits = np.array(logits)
    values = np.array(values)
    print(logits)
    print(chunk.shape)
    print(neighbour.shape)

output = np.pad(attending_chunks.reshape(n, d),((m - 1, 0), (0, 0)), mode='constant')[:n]

output[:m-1] = observation[:m-1]
output[-1] = observation[-1]
#print(output)

-4237097945216324929
839605620643339064
--------
[[[0.32663107 0.32385772 0.16919945 0.30417124 0.35116531]
  [0.77642671 0.76995273 0.40197737 0.72318776 0.83515591]
  [0.48492244 0.47883039 0.25490482 0.44908194 0.51452185]
  [0.36438333 0.36111088 0.18909095 0.33910186 0.39113616]]

 [[0.38367087 0.42822203 0.39612276 0.32777552 0.47958316]
  [0.91255453 1.01715922 0.94148205 0.77843    1.13974018]
  [0.56060603 0.64836097 0.59028527 0.49857447 0.71641796]
  [0.42720214 0.4788569  0.44210532 0.36674087 0.53541341]]]
(4, 2)
(2, 5, 2)
[[[0.37879666 0.33206749 0.15084478 0.4257239  0.35088496]
  [0.34486153 0.30088469 0.13010886 0.38793975 0.31082499]
  [0.48470783 0.42153562 0.17600948 0.54559215 0.42867578]
  [0.53777186 0.47054536 0.20969069 0.60461306 0.49281796]]

 [[0.5554975  0.39718884 0.35555148 0.48259262 0.42035961]
  [0.49804969 0.36132981 0.31118584 0.43134364 0.37934189]
  [0.69271827 0.50759189 0.4254901  0.59864613 0.52997967]
  [0.78388561 0.56371234 0.49704147 0.68017

TypeError: only integer scalar arrays can be converted to a scalar index